In [14]:
import sys
sys.path.insert(0, './algorithms/alpha_beta')
sys.path.insert(0, './algorithms/dqn')
sys.path.insert(0, './algorithms/stockfish')

In [15]:
from algorithms.MyChessBoard import MyChessBoard
from algorithms.alpha_beta.Alpha_Beta import find_move as alpha_beta_find_move
from algorithms.dqn.dqn_bot import find_move as dqn_find_move
from algorithms.dqn.dqn_sf_bot import find_move as dqn_vs_sf_find_move
from algorithms.stockfish.stockfish_bot import find_move as stockfish_find_move

In [16]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten,\
     Input,BatchNormalization, Dropout

In [17]:
STATE_SHAPE = (65, )
NB_ACTIONS = 4096


In [18]:
model_dqn = Sequential()
model_dqn.add(Input((1, ) + STATE_SHAPE))
model_dqn.add(Flatten())
model_dqn.add(Dense(128, activation='relu'))
model_dqn.add(Dropout(0.2))
model_dqn.add(BatchNormalization())
model_dqn.add(Dense(128, activation='relu'))
model_dqn.add(Dropout(0.2))
model_dqn.add(BatchNormalization())
model_dqn.add(Dense(NB_ACTIONS))
model_dqn.add(Activation('linear'))
model_dqn.load_weights('chess_dqn_model.h5')


In [19]:
model_dqn_sf = Sequential()
model_dqn_sf.add(Input((1, ) + STATE_SHAPE))
model_dqn_sf.add(Flatten())
model_dqn_sf.add(Dense(128, activation='relu'))
model_dqn_sf.add(Dropout(0.2))
model_dqn_sf.add(BatchNormalization())
model_dqn_sf.add(Dense(128, activation='relu'))
model_dqn_sf.add(Dropout(0.2))
model_dqn_sf.add(BatchNormalization())
model_dqn_sf.add(Dense(NB_ACTIONS))
model_dqn_sf.add(Activation('linear'))
model_dqn_sf.load_weights('chess_dqn_vs_sf_model.h5')

In [20]:
board = MyChessBoard()

In [21]:
board.convert_board_to_int()

array([[ -40.,  -20.,  -30.,  -50., -900.,  -30.,  -20.,  -40.],
       [ -10.,  -10.,  -10.,  -10.,  -10.,  -10.,  -10.,  -10.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [  10.,   10.,   10.,   10.,   10.,   10.,   10.,   10.],
       [  40.,   20.,   30.,   50.,  900.,   30.,   20.,   40.]])

In [22]:
state = board.get_state()
model_dqn.predict(state.reshape((1, 1) + STATE_SHAPE)).reshape(-1, )

array([-4125.982 , -3346.2021, -3559.5508, ..., -3943.0579, -3572.4902,
       -4092.5286], dtype=float32)

In [23]:
dqn_find_move(board, model_dqn)

Move.from_uci('b1c3')

In [24]:
name = ["Alpha_Beta", "DQN", "DQN_sf", "Stockfish"]
find = [alpha_beta_find_move, dqn_find_move, dqn_vs_sf_find_move, stockfish_find_move]

result = np.zeros((4, 4))
mean_step = np.zeros((4, 4))

In [25]:
num_battle_with_com1 = 50
com1 = 0

com2 = 1
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    step = 0
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board, model_dqn))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board, model_dqn))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


com2 = 2
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


com2 = 3
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


0 vs 1 : 0


In [ ]:
com1 = 1
num_battle_with_com1 = 100

com2 = 2
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)

com2 = 3
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


In [ ]:
com1 = 2
num_battle_with_com1 = 100

com2 = 3
num_step = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            step += 1
            board.push(find[com1](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            step += 1
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
                continue
            if board.is_draw(): 
                done = True
                continue
            board.push(find[com1](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
    num_step.append(step)
mean_step[com1, com2] = np.mean(num_step)


In [ ]:
result